In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import tensorflow_addons as tfa

In [ ]:
sample = "if you want you"

In [ ]:
sample

###### list(set()) -> 안의 값을 나열(set)한 뒤 list로 인덱싱해주는 과정

In [ ]:
idx2char = list(set(sample))

###### {c: i for i, c in enumerate(idx2char)} -> 각각의 값을 dict하여 key와 value로 나누주는 과정

In [ ]:
char2idx = {c: i for i, c in enumerate(idx2char)}

###### x값과 y 값을 인덱스 하나 차이로 두는 이유는 연속적인 단어이기 때문에 이전의 값을 토대로 예측이 필요한 구성이기 때문에

In [ ]:
sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

# 자동으로 원하는 크기 출력
- dic_size -> input size
- rnn_hidden_size = RNN output size
- num_classes = output size
- sequence_length = y_data = [sample_idx[1:]]을 가지므로 -1를 해줌

In [ ]:
dic_size = len(char2idx)
rnn_hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) -1

In [ ]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

###### tf.one_hot를 하면 num_classes만큼 one hot encoding 해줌

In [ ]:
x_one_hot = tf.one_hot(X, num_classes)

In [ ]:
cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

# tf.compat.v1.nn.dynamic_rnn에서 값 제한
- intial_state를 통해 모든 값을 기본화
- intial_state 대신에 sequence_length = [a,b,c]와 같이 원하는 값을 넣으면
- hidden layer층을 지나면서 각각 a, b, c 값을 제외하고 나머지 loss 값 계산을 하지 않아 원하는 loss 값 얻을 수 있음

In [ ]:
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [ ]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y,weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [ ]:
prediction = tf.argmax(outputs, axis=2)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(3000):
    l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
    result = sess.run(prediction, feed_dict={X: x_data})
    result_str = [idx2char[c] for c in np.squeeze(result)]
    print(i, "loss: ", l, "prediction", ''.join(result_str))
sess.close()

# 심화 예제

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import tensorflow_addons as tfa

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
sentence =("if you want to build a ship, dont't drum up people together tocollect wood and don't assign them tasks and work,"
         " but rather teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

###### seq_length = 뽑아내고 싶은 단어의 길이

In [4]:
seq_length = 10
dataX = []
datay = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i + seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    datay.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, dont
23 ship, dont -> hip, dont'
24 hip, dont' -> ip, dont't
25 ip, dont't -> p, dont't 
26 p, dont't  -> , dont't d
27 , dont't d ->  dont't dr
28  dont't dr -> dont't dru
29 dont't dru -> ont't drum
30 ont't drum -> nt't drum 
31 nt't drum  -> t't drum u
32 t't drum u -> 't drum up
33 't drum up -> t drum up 
34 t drum up  ->  drum up p
35  drum up p -> drum up pe
36

In [5]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [6]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = seq_length
batch_size = len(dataX)

In [7]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

In [8]:
x_one_hot = tf.one_hot(X, num_classes)

In [9]:
cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)

C:\Users\since\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:699: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "


In [10]:
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, ,dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\since\anaconda3\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [11]:
prediction = tf.argmax(outputs, axis=2)

In [12]:
weights = tf.ones([batch_size, seq_length])
sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y,weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 오류가 나는 이유? 
- logits 부분에서 활성함수를 거친 output 값을 넣어주기 때문에 오류가 나므로 활성함수를 거치지 않은 output 값을 넣어주면 됨
- 뒤 부분은 새로운 장에 다시 시작

In [14]:
for i in range(500):
    l, _ = sess.run([loss, train], feed_dict={X: dataX, Y: datay})
    result = sess.run(prediction, feed_dict={X:dataX})
    result_str = [char_set[c] for c in np.squeeze(result)]
    print(i, "loss: ", l, "prediction", ''.join(result_str))
sess.close()

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: assertion failed: [Expected shape for Tensor rnn/sequence_length:0 is ] [170] [ but saw shape: ] [1]
	 [[node rnn/Assert/Assert (defined at \AppData\Local\Temp/ipykernel_4744/1064150691.py:2) ]]
  (1) Invalid argument: assertion failed: [Expected shape for Tensor rnn/sequence_length:0 is ] [170] [ but saw shape: ] [1]
	 [[node rnn/Assert/Assert (defined at \AppData\Local\Temp/ipykernel_4744/1064150691.py:2) ]]
	 [[Mean/_25]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'rnn/Assert/Assert':
  File "\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_ins